<div>
<img src="images.png" width="100%"/>
</div>

<h1><center>TOPIC FOR THE TEMPLATE</center></h1>

## Pre-requisite and data sources

Here define the prerequisites for the notrbooks. See the guideline for more details.

## Table of Contents

Define the table of contents and update whenever new content is added to the page.


- [Introduction](#Introduction)

- [Objectives](#Objectives)

- [Library imports](#Library-imports)

- [Access and authentication](#Data-access-and-authentication)

- [Data download](#Data-download)

- [Data processing](#Data-reading-and-pre\-processing-such-as-filter\,-area-definition-and-data-sampling)

- [Data analysis](#Data-analysis/evaluation-and-plotting)

- [Results visualisation](#Results-visualisation)

- [Save](#Save)

- [Conclusion](#Conclusion)

- [Reference](#References)

## Introduction

Provide the introduction to the notebook here.

## Objectives

Define the objectives of the notebooks

## Library imports

As a reminder, the libraries are to be imported in the order of 

- Standard library
- Related third party imports
- Local application of library specific imports

Remember to leave a space between each library grouping for organisational purposes only

In [ ]:
import os
from datetime import timedelta, datetime, date

import numpy as np
import matplotlib.pyplot as plt
import netCDF4 as nc
import cartopy as cart
import ipywidgets as widgets
import math
import xarray as xr
from netCDF4 import num2date

from hsaf_data_access import HSAFDataAccess as data_access

To carry out the exercise, we need to create auxiliary folders by name data and output where the downloaded data and the results of our processing will be stored respectively. 

In [ ]:
# This is an auxilliary code
if not os.path.exists('output'):
    os.makedirs('output')
if not os.path.exists('data'):
    os.makedirs('data')
else:
    for f in os.listdir('data'):
        os.remove(os.path.join('data', f))

## Data access and authentication
In this instance, the user credentials (username and password) needed for accessing the FTP server will be required. Thus, the user has to be pre-informed on the need to provide their own credentials (or ad-hoc login info created for training purposes) in order for the required data to be downloaded from the server. A non-registered user needs to follow the link below to create an account.

   [Link to create an account on H-SAF Website](https://hsaf.meteoam.it/ "Follow this link if you don't have an H-SAF account")

The user credentials are to be entered in a widget available by running the code cell below.  
In addition, a date picker tool is available for specifying the period in which the analysis is to be made.

In [ ]:
work_dir = os.getcwd()
os.chdir(work_dir)
storedir = work_dir + '/data/'

datestart = widgets.DatePicker(description='Start Date', disabled=False, value = date.today(), max = date.today())
dateend = widgets.DatePicker(description='End Date', disabled=False, value = date.today(), max = date.today())
display(widgets.HBox([datestart, dateend]))


username = widgets.Text(description='Username:', disabled=False)
psw = widgets.Password(description='Password:', disabled=False)
display(widgets.HBox([username, psw]))

## Data download
The values of the username, password and date picker widgets will be supplied as argument for the download function. However, data will be downloaded for only the days in the date range in which data is available. For days in which data is not available, feedback message is printed to the end-user.

In [ ]:
try:
    data_access.download_h64(username.value, psw.value, datestart.value, dateend.value, storedir, 'h64')
except:
    print('Incorrect parameters in the \'Data access and authentication\' section')

## Data reading and pre-processing such as filter, area definition and data sampling

The variables x and y in this section indicate the coordinates of the bounding box around defining the area of study.

#### Area definition
Specify the bounding box of the study area in the numpy array in the order of 
* lower left
* lower right
* upper right
* upper left

In [ ]:
x = np.array([20, 30, 30, 20, 20])
y = np.array([35, 35, 45, 45, 45])

#### Data unwrapping
The downloaded data are in a gz wrap and here we need to extract the content of the zip file in the data folder.  
Also, we need to remove empty files after the extraction.

In [ ]:
data_access.extract_gz_file(storedir)

# Remove empty files
for f, ff, in enumerate(os.listdir(storedir)):
    if os.stat(ff).st_size == 0:
        os.remove(ff)

                            
filelist = os.listdir(storedir)                    
filelist.sort()
plt_dim = math.ceil(math.sqrt(len(filelist)))

In [ ]:
# Redefine the initial and ending date based only on the days data is available
datestart = datetime(int(filelist[0][4:8]), int(filelist[0][8:10]), int(filelist[0][10:12]))
dateend = datetime(int(filelist[-1][4:8]), int(filelist[-1][8:10]), int(filelist[-1][10:12]))

#### Data Reading and plotting
The extracted data are in a netCDF4 format and we need to read them here.
We start by preparing the axes on which the plots are going to be made, read and process the data and then plot on the axes.

In [ ]:
# Number of subplots according to the number of files in the directory
if len(filelist) > 1:
    fig, axs = plt.subplots(ncols=plt_dim, nrows=plt_dim, subplot_kw={'projection': cart.crs.PlateCarree()})
    axs = axs.ravel()  
    
else:
    fig, axs = plt.subplots(ncols=1, nrows=1, squeeze=False, subplot_kw={'projection': cart.crs.PlateCarree()})
    
# The title of the figure based on the difference between the start and end dates
if dateend > datestart:
    if dateend.year > datestart.year:
        fig.suptitle(datestart.date().strftime("%d/%m/%Y") + ' - ' + dateend.date().strftime("%d/%m/%Y") + 
                     ' H60 rainfall', fontsize=18)
    elif dateend.month > datestart.month:
        fig.suptitle(datestart.date().strftime("%d/%m") + ' - ' + dateend.date().strftime("%d/%m/%Y") + 
                     ' H60 rainfall', fontsize=18)
    else:
        fig.suptitle(datestart.date().strftime("%d") + ' - ' + dateend.date().strftime("%d/%m/%Y") +
                      ' H60 rainfall', fontsize=18)
else:
    fig.suptitle(datestart.date().strftime("%d/%m/%Y") + ' H60 rainfall',
                  fontsize=18)
    
axs=axs.flatten()
Rain_event = []
outfilename = 'H64_Rainfall_Event_BG.nc'

for n, element in enumerate(filelist):
    ds = nc.Dataset(element, mmap=False)
    lon = ds['lon'][:]
    lat = ds['lat'][:]
    P_h64 = ds['acc_rr'][:]
    lat_h64, lon_h64 = np.meshgrid(lat, lon)
    lat_h64 = np.ravel(lat_h64)
    lon_h64 = np.ravel(lon_h64)
    IN = data_access.in_polygon(lon_h64, lat_h64, x, y)
    lon_h64 = lon_h64[IN]
    lat_h64 = lat_h64[IN]
    P_h64 = np.transpose(P_h64)
    P_h64 = np.ma.getdata(P_h64)
    P_h64 = np.where(P_h64 < 1, np.NaN, P_h64)
    P_h64 = np.ravel(P_h64)
    P_h64 = P_h64[IN]
    Rain_event = np.append(Rain_event, P_h64, axis=0)
    if len(filelist) == 1:
        im = axs[n].scatter(lon_h64, lat_h64, c=P_h64, marker=',', vmin=0, vmax=100)
        axs[n].set_title(element[10:12]+'/'+element[8:10]+'/'+element[4:8]+' rainfall', y=-0.1)
        cbar = fig.colorbar(im, ax=axs, aspect=30, shrink=0.70)
        cbar.ax.set_xlabel('mm/day', loc='left')
    else:
        bbox = axs[n].get_window_extent().transformed(fig.dpi_scale_trans.inverted())
        axs_width = bbox.width
        im = axs[n].scatter(lon_h64, lat_h64, c=P_h64, marker=',', vmin=0, vmax=100)
        axs[n].set_title(element[10:12]+'/'+element[8:10]+'/'+element[4:8], pad=0.2, fontsize=axs_width*10)
        plt.subplots_adjust(wspace=axs_width, hspace=axs_width)
         
        if n == len(filelist)-1:
            cbar = fig.colorbar(im, ax=axs)
            cbar.set_label('mm/day', loc='center')
    data_access.add_border(axs[n])
    
    ds.close()

# Delete those axes that have no plot on them
for ax in axs.flatten():
    if not ax.has_data():
        fig.delaxes(ax)

os.chdir(work_dir)
plt.savefig('output/H64_Rainfall_Event_BG.png', dpi=300)
plt.close()
Rain = Rain_event.reshape((len(filelist), len(lon_h64)))

plt.figure(1)
Rain_plot = np.nansum(Rain, 0)
Rain_plot = np.where(Rain_plot < 1, np.NaN, Rain_plot)

bckg = plt.axes(projection=cart.crs.PlateCarree())
data_access.add_border(bckg)

im2 = plt.scatter(lon_h64, lat_h64,c=Rain_plot, marker=',', vmin=0, vmax=200)
plt.title('Total event rainfall', fontsize=16)
cbar2 = plt.colorbar(shrink=0.66)
cbar2.set_label('mm')
plt.savefig('output/H64_Total_Rainfall_BG.png', dpi=300)
plt.close()


## Data analysis/evaluation
Given that the plot has been made, analysis on it needs to be made. Other explanation for the variation of the variable are also made in this section.

## Results visualisation
Show the plot in this section. Please note that a square grid is used for making the plots and axes, which do not have plots on them, are deleted. This is due to the dynamic nature of the subplot dimensions, which depend on the number of days analysis is being made. 

![alt_text](output/H64_Rainfall_Event_BG.png)

## Save

In [ ]:
# Save the data as netCDF4 file
data_access.to_netCDF('Rainfall', 'mm', outfilename, Rain, lat_h64, lon_h64, datestart, dateend)

In [ ]:
# Convert the netCDF4 file to CSV
ds = xr.open_dataset(outfilename, decode_times=True)
df = ds.to_dataframe()

df.to_csv('output/output.csv')
ds.close()

# Get data for a specific region using shapefile
# data_access.cut_by_shp('specify path of shapefile', 'specify path of netCDF4 file')


## Conclusion
Give a short concluding statement to the work.

## References
Provide list of all reference materials necessary for understanding the topic at hand as well as those used to write the document.

[Jump to TOC](#Table-of-Contents)